NOTE: THIS IS A TEMPORARY HIGHLY-ITERATIVE RESEARCH DOCUMENT. TEXT WILL BE ADDED LATER.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("sap_storing_data_hu_project.csv")
df

In [ ]:
fh_tijdstip = df["stm_fh_ddt"]
stor_eind = df["stm_sap_storeind_ddt"]
stm_sap_meld_ddt = df["stm_sap_meld_ddt"]

In [ ]:
fh_tijdstip = pd.to_datetime(fh_tijdstip, errors="coerce")
stor_eind = pd.to_datetime(stor_eind, errors="coerce")
stm_sap_meld_ddt = pd.to_datetime(stm_sap_meld_ddt, errors="coerce")

twominutes = pd.to_timedelta(2, unit="m")

In [ ]:
new_df = pd.DataFrame({"fh_ddt": fh_tijdstip,
                       "se_ddt": stor_eind,
                       "meld_ddt": stm_sap_meld_ddt}).dropna()

new_df.shape  # 1/3rd is dropped from nas

In bijna de helft van de gevallen liggen 'storing eind' en 'functieherstel' binnen 2 minuten van elkaar.
Storing eind (- 2min) is in de helft van de gevallen groter dan functieherstel.

In [ ]:
(np.abs(new_df["fh_ddt"] - new_df["se_ddt"]) < twominutes).sum()

In [ ]:
((new_df["se_ddt"] - twominutes) > new_df["fh_ddt"]).sum()

Waarschijnlijk is functieherstel tijdstip - aankomst aannemer het targetvariabele. Hier hebben we tijdstip en datum apart van, of schijnbaar een datetime, maar in de data dictionairy staat die als ?.


In [ ]:
df["stm_aanntpl_dd"].isna().sum()

In [ ]:
df["stm_aanntpl_tijd"].isna().sum()

In [ ]:
df["stm_aanntpl_ddt"].isna().sum()

In [ ]:
df[np.logical_not(df["stm_aanntpl_ddt"].isna())]["stm_aanntpl_ddt"]

In [ ]:
wonky_format = df["stm_aanntpl_dd"] + " " + df["stm_aanntpl_tijd"]

df_aan = pd.DataFrame({"t": pd.to_datetime(df["stm_aanntpl_tijd"], errors="coerce"),
                       "d": pd.to_datetime(df["stm_aanntpl_dd"], errors="coerce"),
                       "dt": pd.to_datetime(df["stm_aanntpl_ddt"], errors="coerce"),
                       "dt_wonky": pd.to_datetime(wonky_format, errors="coerce")}).dropna()

df_aan.shape

In [ ]:
df_aan

In [ ]:
(df_aan["dt"] == df_aan["dt_wonky"]).sum()

We kunnen dus als het goed is de ddt kolom dus gewoon gebruiken...

Dan nog even kijken of er idd genoeg nuttige entries zijn voor functieherstel - arrival monteur

In [ ]:
possible_target = new_df["fh_ddt"] - df_aan["dt"]

In [ ]:
possible_target.shape

In [ ]:
possible_target.isna().sum()

In [ ]:
possible_target.dropna(inplace=True)

In [ ]:
target_secs = possible_target.dt.total_seconds()
(target_secs == 0).sum()

In [ ]:
target_secs = target_secs.loc[(target_secs > 1) | (target_secs < -1)]

In [ ]:
(target_secs == 0).sum()

In [ ]:
target_secs

In [ ]:
np.max(target_secs)

In [ ]:
def outliers(s: pd.Series) -> pd.Series:
    q1, q3 = s.quantile(.25), s.quantile(.75)
    iqr = q3 - q1
    return s.loc[(s > q1 - (3 * iqr)) & (s < q3 + (3 * iqr))]

target_secs = outliers(target_secs)

In [ ]:
target_secs.hist(bins=50)

In [ ]:
plt.boxplot(target_secs)

In [ ]:
target_secs.shape

Als dit inderdaad een goed targetvariabele is, blijven er 560K van de 890K rows over.